# Projeto Agile Tech - Jonas Honorato


## Workflow
  1 - Análise Exploratória dos Dados

## 1 - Análise Exploratória dos Dados
Nesta etapa do estudo de caso, vamos entender que informações cada arquivo csv armazena.

In [194]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
import joblib # Para salvar o modelo


In [195]:
df_clientes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projeto Agile Tech/clients.csv")
df_chat = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projeto Agile Tech/chat_history.csv")
df_client_conditions = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projeto Agile Tech/client_conditions.csv")
df_seed_chapter = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projeto Agile Tech/seed_ciap_chapters.csv")
df_seed_components = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projeto Agile Tech/seed_ciap_components.csv")
df_seed_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projeto Agile Tech/seed_ciap_raw.csv")

### Tabela de Clientes

In [196]:
df_clientes.head()

,client_id,chat_message_id,idade,genero,cidade,plano_saude,ultima_consulta
0,1,XsuEERTR-qmASCASD,58,Masculino,Nascimento,Sim,2025-01-26
1,2,XsuEA5rR-qmPzN3L,65,Feminino,Cavalcante da Prata,Sim,2024-06-10
2,3,aPgkognhKRQ9Z9B_,32,Feminino,Viana de Cardoso,Sim,2024-07-14
3,4,QNKhrEx4sAYOj0rh,61,Feminino,Caldeira do Oeste,Sim,2024-03-24


In [197]:
# Verificando quantidade de clientes
print(df_clientes.shape)

(4, 7)


**Há 4 clientes na base de dados**

### Tabela de chat

In [198]:
df_chat.head()

,message_text,sender,created_date_brt,chat_channel_id,updated_date_brt
0,me conte um pouco sobre como é a sua rotina co...,user,2024-10-22T19:57:39.503000,XsuEA5rR-qmPzN3L,2024-10-22T19:57:53
1,um regime,client,2024-10-22T19:40:26.877000,XsuEA5rR-qmPzN3L,2024-10-22T19:51:38
2,"sem problemas! no momento, posso te ajudar de ...",user,2024-10-22T18:53:40.416000,XsuEA5rR-qmPzN3L,2024-10-22T19:40:16
3,cancela,client,2024-10-22T18:33:53.202000,XsuEA5rR-qmPzN3L,2024-10-22T18:50:33
4,acredito que tenha mandado esse comprovante po...,user,2024-10-22T17:57:42.488000,XsuEA5rR-qmPzN3L,2024-10-22T18:33:42


In [199]:
# Verificando quantas mensagens temos na base
df_chat.shape

(115, 5)

**Há 115 mensagens na base de dados**

### Tabela de condição de clientes

In [200]:
df_client_conditions.head()

,client_id,condition,cid10
0,2,doença diverticular do intestino,k57
1,2,outra embolia e trombose venosas,i82
2,3,Distúrbios do início e da manutenção do sono,g47
3,3,obesidade,e66


In [201]:
df_client_conditions.shape

(4, 3)

**Há 04 possíveis condições de clientes**

### Tabelas Seed Ciap

In [202]:
df_seed_chapter.head()

,letra,descrição
0,A,Geral e não-específico
1,B,"Sangue, órgãos hematopoiéticos e linfáticos (b..."
2,D,Aparelho digestivo
3,F,Olhos
4,H,Ouvidos


In [203]:
df_seed_components

,componente,descrição
0,1,Componente de queixas e sintomas
1,2,Componente de procedimentos diagnósticos e pre...
2,3,"Componente de medicações, tratamentos e proced..."
3,4,Componente de resultados de exames
4,5,Componente administrativo
5,6,Componente de acompanhamento e outros motivos ...
6,7,Componente de diagnósticos e doenças


In [204]:
df_seed_raw.head()

,codigo,componente,titulo original,titulo leigo,CIDS10 possíveis,CID10 mais frequente,definição,critérios de inclusão,critérios de exclusão,considerar,nota,Unnamed: 11
0,A01,1.0,Dor generalizada /múltipla,Dor generalizada /múltipla,R52,R52,NaN,"dor crônica generalizada, dores múltiplas",dor não-específica A29,NaN,NaN,NaN
1,A02,1.0,Arrepios/ calafrios,Arrepios/ calafrios,R68.8,R68.8,NaN,"calafrios, tremores",febre A03,NaN,NaN,NaN
2,A03,1.0,Febre,Febre,R50,R50,NaN,pirexia,"com exantemas A76, exaustão pelo calor ou golp...",NaN,NaN,NaN
3,A04,1.0,Debilidade/cansaço geral/fadiga,Debilidade/cansaço geral/fadiga,"G93.3, R53",R53,NaN,"de fadiga crônica, exaustão, fadiga, lassidão,...","sentir-se paciente, má-disposição A05; modorra...",NaN,NaN,NaN
4,A05,1.0,Sentir-se doente,Sentir-se doente,R53,R53,NaN,mal-estar,"senescência, senilidade P05, caquexia T08, mal...",NaN,NaN,NaN


## Insights até o momento
- as tabelas df_clientes e df_chat se relacionam
- a tabela df_client_conditions tem uma provável variável alvo
- a coluna cid10 da tabela df_client_conditions aparece em df_seed_raw, podemos utilizar CDIS10 possivel,CDI10 mais frequente,criterio de inclusao e de exclusao da tabela df_seed_raw para treinar  o modelo.
- é possível agrupar as mensagens de cada client por ordem de data para conseguir o texto completo.

## Pré Processamento e Feature Engineering
Nesta fase iremos preparar os dados para treinamento do modelo

### Agrupando as mensagens por cliente

In [205]:
try:
    df_chat['created_date_brt'] = pd.to_datetime(df_chat['created_date_brt'])
    print("Coluna 'created_date_brt' convertida para datetime com sucesso.")
except Exception as e:
    print(f"Erro ao converter 'created_date_brt': {e}")
    print("Por favor, verifique o formato das datas. Continuando mesmo assim...")

Coluna 'created_date_brt' convertida para datetime com sucesso.


In [206]:
df_chat_ordenado = df_chat.sort_values(by='created_date_brt')

In [207]:
# 3. Criar uma função para formatar a conversa de cada grupo
def formatar_conversa(df_grupo):
    """
    Recebe um DataFrame de um único chat e formata
    as mensagens com o 'sender' e uma quebra de linha.

    CORRIGIDO: Adiciona .fillna('') para tratar valores nulos (NaN)
    e os transforma em string vazia.
    """

    # Preenche NaNs (nulos) com string vazia ANTES de concatenar
    senders_str = df_grupo['sender'].fillna('')
    messages_str = df_grupo['message_text'].fillna('')

    # Agora a concatenação é segura, pois só temos strings
    mensagens_formatadas = senders_str + ": " + messages_str

    # Junta todas as mensagens formatadas com uma quebra de linha (" \n ")
    return " \n ".join(mensagens_formatadas)

In [208]:
# 4. Agrupar por 'chat_channel_id' e APLICAR a nova função
df_X = df_chat_ordenado.groupby('chat_channel_id').apply(formatar_conversa, include_groups=False).reset_index()

In [209]:
# Renomear a coluna message_text para texto completo
df_X = df_X.rename(columns={0: 'texto_completo'})

In [210]:
df_X.head()

,chat_channel_id,texto_completo
0,QNKhrEx4sAYOj0rh,": boa tarde! \n user: boa tarde, sra. jacira! ..."
1,XsuEA5rR-qmPzN3L,: sim \n : como é o procedimento? \n user: bom...
2,aPgkognhKRQ9Z9B_,: boa tarde \n : então estou com uma unha encr...


In [211]:
# Exemplo de chat completo
df_X.iloc[0,1]

': boa tarde! \n user: boa tarde, sra. jacira! vou te explicar um pouco sobre o nosso canal \n user: por aqui estou a disposição para te ajudar a cuidar de sua saúde. por exemplo, caso queira melhorar seu estilo de vida, posso te indicar hábitos saudáveis em relação a alimentação, atividade física ou sono. caso tenha queixas clínicas, como sintomas gripais, cuidados com feridas, doenças crônicas ou outros, também posso te ajudar, seja direcionando você para o canal correto, seja orientando cuidados mais específicos. tudo que for relacionado a sua saúde você pode me perguntar \n : na verdade jacira é  minha mãe. \n : ela não sabe mexer  no telefone \n : este serviços  são  cobrados? \n : ela tem uma boa alimentação,  mas tem problema nos ossos, osteoporose, artrose etc.. e também tem fibromioalgia \n user: nosso acompanhamento aqui pelo whatsapp não possui custo adicional \n user: qual seu nome? por gentileza \n : tania \n user: tania, obrigada por compartilhar comigo um pouco do histór

In [212]:
# Organizando variável target:
df_y_raw = pd.crosstab(
    df_client_conditions['client_id'],
    df_client_conditions['cid10']
)

In [213]:
# 2. Binarizar o resultado
# Garante que só tenhamos 0 (ausente) ou 1 (presente)
df_y = df_y_raw.astype(bool).astype(int)

In [214]:
# 3. Resetar o índice
# Transforma o 'client_id' (que virou o índice) de volta em uma coluna
df_y = df_y.reset_index()

In [215]:
df_y.head()

cid10,client_id,e66,g47,i82,k57
0,2,0,0,1,1
1,3,1,1,0,0


In [216]:
# Construindo df de treino (feature e target)

# Df com features(texto e cliente)
df_com_texto_e_id = pd.merge(
    df_X,
    df_clientes,
    left_on='chat_channel_id',
    right_on='chat_message_id', # A chave que você confirmou que liga os dois
    how='inner'
)

In [217]:
df_com_texto_e_id.head()

,chat_channel_id,texto_completo,client_id,chat_message_id,idade,genero,cidade,plano_saude,ultima_consulta
0,QNKhrEx4sAYOj0rh,": boa tarde! \n user: boa tarde, sra. jacira! ...",4,QNKhrEx4sAYOj0rh,61,Feminino,Caldeira do Oeste,Sim,2024-03-24
1,XsuEA5rR-qmPzN3L,: sim \n : como é o procedimento? \n user: bom...,2,XsuEA5rR-qmPzN3L,65,Feminino,Cavalcante da Prata,Sim,2024-06-10
2,aPgkognhKRQ9Z9B_,: boa tarde \n : então estou com uma unha encr...,3,aPgkognhKRQ9Z9B_,32,Feminino,Viana de Cardoso,Sim,2024-07-14


In [218]:
# dataframe de treino completo, com feature e rotulo
df_treino = pd.merge(
    df_com_texto_e_id,
    df_y,
    on='client_id', # A chave comum entre os dois
    how='inner'
)

df_treino.head()

,chat_channel_id,texto_completo,client_id,chat_message_id,idade,genero,cidade,plano_saude,ultima_consulta,e66,g47,i82,k57
0,XsuEA5rR-qmPzN3L,: sim \n : como é o procedimento? \n user: bom...,2,XsuEA5rR-qmPzN3L,65,Feminino,Cavalcante da Prata,Sim,2024-06-10,0,0,1,1
1,aPgkognhKRQ9Z9B_,: boa tarde \n : então estou com uma unha encr...,3,aPgkognhKRQ9Z9B_,32,Feminino,Viana de Cardoso,Sim,2024-07-14,1,1,0,0


**Ainda há colunas inúteis para o modelo no df_treino, vamos excluí-las**

**Queremos que o modelo se baseie apenas no texto das mensagens**

In [219]:
colunas_alvo = list(df_y.columns.drop('client_id'))

In [220]:
colunas_para_manter = ['client_id', 'texto_completo'] + colunas_alvo

In [221]:
df_treino_limpo = df_treino[colunas_para_manter]

In [222]:
df_treino_limpo.head()

,client_id,texto_completo,e66,g47,i82,k57
0,2,: sim \n : como é o procedimento? \n user: bom...,0,0,1,1
1,3,: boa tarde \n : então estou com uma unha encr...,1,1,0,0


### Observação:

  - O df de treino contém apenas 2 exemplares, por isso, é estatisticamente inviável treinar um modelo com esses dados.

  - Treinaremos o modelo como prova de conceito de um pipeline de ML
  
  - Construiremos uma API com Fast API

## Treinamento do Modelo

In [223]:
# Nosso dataset final, com 2 linhas
df_treino_final = df_treino_limpo.copy()

print("Iniciando o Passo 4: Treinamento (Prova de Conceito)...")
print(f"Treinando com {len(df_treino_final)} amostras (conforme descoberto na análise).")

# 1. Separar X e y do nosso dataset limpo
X_texto = df_treino_final['texto_completo']

# Pega os nomes das colunas de alvo (ex: ['e66', 'g47', 'i82', 'k57'])
# (Presumindo que 'df_y' do Passo 2 ainda está na memória)
colunas_alvo = list(df_y.columns.drop('client_id'))
y_labels = df_treino_final[colunas_alvo]

print(f"Features (X): {len(X_texto)} textos.")
print(f"Alvos (y): {len(y_labels)} conjuntos de rótulos.")

# --- Pipeline de Modelagem ---

# 2. Inicializar o Vetorizador (TF-IDF)
# Ele vai transformar o texto em números
vectorizer = TfidfVectorizer()

# 3. Inicializar o Modelo
# Usamos um classificador simples (Regressão Logística)
# envolvido por um MultiOutputClassifier (para lidar com as 4 colunas de alvo)
base_model = LogisticRegression(random_state=42)
multi_output_model = MultiOutputClassifier(base_model, n_jobs=-1)

# 4. Treinar o Vetorizador e transformar o X
# .fit_transform() aprende o vocabulário e transforma o texto
X_vetorizado = vectorizer.fit_transform(X_texto)

# 5. Treinar o Modelo
# .fit() treina o classificador nos dados vetorizados
multi_output_model.fit(X_vetorizado, y_labels)

print("Modelo treinado com sucesso (em 2 amostras).")

# --- Salvando os artefatos ---
# Precisamos salvar AMBOS: o vetorizador e o modelo treinado.

# Salva o vetorizador
joblib.dump(vectorizer, 'vectorizer.joblib')

# Salva o modelo
joblib.dump(multi_output_model, 'model.joblib')

print("Artefatos 'vectorizer.joblib' e 'model.joblib' salvos com sucesso.")

Iniciando o Passo 4: Treinamento (Prova de Conceito)...
Treinando com 2 amostras (conforme descoberto na análise).
Features (X): 2 textos.
Alvos (y): 2 conjuntos de rótulos.
Modelo treinado com sucesso (em 2 amostras).
Artefatos 'vectorizer.joblib' e 'model.joblib' salvos com sucesso.
